In [1]:
from typing import Dict, List
from collections import defaultdict
import random

from autocorrect import Speller
from pymorphy2 import MorphAnalyzer
import pandas as pd
import numpy as np
import re
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 80)

## Читаем датасеты и выполняем обзор имеющихся данных

In [2]:
df_train = pd.read_csv('train_dataset_train.csv')
df_test = pd.read_csv('test_dataset_test.csv')

In [3]:
df_train.head()

,id,Текст Сообщения,Тематика,Ответственное лицо,Категория
0,2246,Помогите начальник Льговского рэс не реагирует...,"Нарушения, связанные с содержанием электросети...",Администрация Льговского района,3
1,380,<p>По фасаду дома по адресу ул. Урицкого 22 пр...,Аварийные деревья,Администрация города Курска,3
2,2240,Агресивные собаки. На радуге там стая из подро...,Безнадзорные животные,Администрация города Курска,1
3,596,<p>На пересечении &nbsp;улиц Сосновская и Бере...,Нескошенная сорная растительность в местах общ...,Комитет дорожного хозяйства города Курска,3
4,1797,<p style=`text-align:justify;`><span style=`ba...,Аварийные деревья,Комитет городского хозяйства города Курска,3


In [4]:
df_test.head()

,id,Текст Сообщения,Тематика,Ответственное лицо
0,843,<p>Здравствуйте. На улице Мира &nbsp;было заме...,Неработающее наружное освещение,Администрация Курчатовского района
1,1422,<p>Уже вторую неделю не горит уличное освещени...,Неработающее наружное освещение,Комитет жилищно-коммунального хозяйства города...
2,2782,Не работает освещение во дворе дома 11а по Эне...,Неработающее наружное освещение,Комитет жилищно-коммунального хозяйства города...
3,2704,После покоса сорной растительности на газоне м...,Неудовлетворительная уборка улиц и тротуаров,Администрация Центрального округа города Курска
4,1,<p>Прошу принять меры к водителю маршрута 263:...,Неудовлетворительный внешний вид (поведение) в...,Администрация города Курска


## Оценка влияния полей на категорию обращения

In [5]:
len(set(df_train['Тематика']))

161

Убедимся, что одной тематике соответствует одна и та же категория

In [6]:
df_train_grp_theme = df_train.groupby('Тематика')

In [7]:
df_train_grp_theme = df_train_grp_theme['Категория'].agg([('cat_min', 'min'), ('cat_max', 'max'), ('theme_count', 'count')])

In [8]:
np.sum(df_train_grp_theme.cat_min != df_train_grp_theme.cat_max)

0

В то же время можно заметить наличие большого количества тем, присутствующих только в одной записи.

In [9]:
df_train_grp_theme.theme_count.sort_values(axis=0, ascending=True).head(30)

Тематика
Нарушения в вопросах оплаты коммунальных услуг в микрорайонах ИЖС                                                                       1
Нарушения, связанные с обеспечением детским питанием                                                                                    1
Неудовлетворительное качество работы преподавателей и другого персонала учреждения дополнительного образования                          1
Нарушение графика движения при осуществлении внутриобластных междугородных перевозок                                                    1
Нарушение графика движения при осуществлении муниципальных (пригородных) перевозок                                                      1
Недостаточное обеспечение кадрами учреждений здравоохранения                                                                            1
Отказ в вакцинации                                                                                                                      1
Недостаточное количество 

Это означает, что уже при кросс-валидации мы столкнемся с трудностью при попытке определить категорию по теме: 
подходящей темы в обучающем датасете может просто не оказаться.
Вместо этого мы будем искать не точно такую же тему, а похожую.
Но и в этом случае можно столкнуться с ситуацией, когда похожей методики не существует. 
Поэтому имеет смысл посмотреть и на другие поля.

In [7]:
pd.crosstab(df_train['Ответственное лицо'], df_train['Категория'])

Категория,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Ответственное лицо,,,,,,,,,,,,,,,,,
Администрация Беловского района,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
Администрация Большесолдатского района,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0
Администрация Глушковского района,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
Администрация Горшеченского района,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1
Администрация Железногорского района,5,0,0,6,0,0,0,1,0,0,0,0,0,0,0,0,0
Администрация Железнодорожного округа города Курска,3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3
Администрация Золотухинского района,3,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1
Администрация Касторенского района,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
Администрация Конышевского района,0,0,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0


In [10]:
df_train['Тип ответственного'] = df_train['Ответственное лицо'].apply(lambda x: 'Администрация' if x[:13] == 'Администрация' 
                                                                     else 'Комитет' if x[:7] == 'Комитет' else 'Другое')

In [11]:
pd.crosstab(df_train['Тип ответственного'], df_train['Категория'])

Категория,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Тип ответственного,,,,,,,,,,,,,,,,,
Администрация,280,11,1,369,0,9,5,13,53,0,12,11,0,6,1,2,76
Другое,3,0,0,242,2,0,0,0,19,0,1,1,1,1,0,0,3
Комитет,195,14,2,343,106,3,5,14,67,5,35,7,0,4,3,5,70


## Подготовка и преобразование данных

In [12]:
# Функции для обработки текста

RU_LETTERS = re.compile('[а-яё]+')
CLEAN_HTML = re.compile('<.*?>')
spell = Speller('ru')
morph = MorphAnalyzer()

# Удяляем html-теги, в нижний регистр, оставляем только русские буквы. Разбиваем на токены.
# Удаляем слова короче 3-х символов как неинформативные.
def format_text(raw_text: str) -> str:
    # Убираем html-теги
    txt = re.sub(CLEAN_HTML, '', raw_text)
    
    # Приводим к нижнему регистру
    txt = txt.lower()
    
    # Оставляем только слова из русских букв
    txt = " ".join(RU_LETTERS.findall(txt))
    
    # Удаляем неинформативные слова длиной менее 2-х букв.
    words_filtered = []
    for word in txt.split():
        if len(word) > 2:
            words_filtered.append(word)
    return ' '.join(words_filtered)


# Приводим все слова к нормальной форме. Удаляем неинформативные части речи: предлоги, союзы,частицы, междометия.
def format_norm(txt: str) -> str:
    words = []
    for word in txt.split():
        m = morph.parse(word)[0]
        if m.tag.POS not in ('PREP', 'CONJ', 'PRCL', 'INTJ'):
            words.append(m.normal_form)
    return ' '.join(words)
    

In [13]:
# Подготовка данных для заданного датасета.

def format_df_inplace(df: pd.DataFrame) -> None:
    # "Причёсываем" Текст Сообщения.
    df['Текст Сообщения'] = df['Текст Сообщения'].apply(format_text)
    # Исправляем опечатки.
    df['Текст Сообщения'] = df['Текст Сообщения'].apply(spell)
    # Приводим к нормальной форме
    df['Текст Сообщения'] = df['Текст Сообщения'].apply(format_norm)
    
    # Тематику преобразовываем в набор токенов.
    df['Тематика'] = df['Тематика'].apply(format_text)
    # Нормализованные слова помещаем в отдельное поле, так как исходная Тематика нам пригодится
    df['Тематика Норм'] = df['Тематика'].apply(format_norm)
    
    # Ответственное лицо разбиваем на типы: Администрация, Комиссия, Другое
    df['Тип ответственного'] = df['Ответственное лицо'].apply(lambda x: 'Администрация' if x[:13] == 'Администрация' 
                                                             else 'Комитет' if x[:7] == 'Комитет' else 'Другое')

In [18]:
# Еще раз читаем исходные данные на случай, если в ходе экспериментов в них уже что-то было поломано

df_train = pd.read_csv('train_dataset_train.csv')
df_test = pd.read_csv('test_dataset_test.csv')

format_df_inplace(df_train)
format_df_inplace(df_test)

# Так как подготовка знимает много времени, сохраним результаты чтобы в следующий раз сразу загрузить уже подготовленные данные.
df_train.to_csv('train_dataset_prepared.csv')
df_test.to_csv('test_dataset_prepared.csv')

In [19]:
df_train = pd.read_csv('train_dataset_prepared.csv', index_col=0)
df_test = pd.read_csv('test_dataset_prepared.csv', index_col=0)


In [20]:
df_train.head()

,id,Текст Сообщения,Тематика,Ответственное лицо,Категория,Тематика Норм,Тип ответственного
0,2246,помочь начальник льговский рэс реагировать жал...,нарушения связанные содержанием электросети ка...,Администрация Льговского района,3,нарушение связанный содержание электросеть кач...,Администрация
1,380,фасад дом адрес урицкий проходить труба газовы...,аварийные деревья,Администрация города Курска,3,аварийный дерево,Администрация
2,2240,агрессивный собака радуга там стая подрасти ще...,безнадзорные животные,Администрация города Курска,1,безнадзорный животное,Администрация
3,596,пересечение улица сосновый береговой завалить ...,нескошенная сорная растительность местах общег...,Комитет дорожного хозяйства города Курска,3,нескошенный сорный растительность место общий ...,Комитет
4,1797,рядом дом улица светлый придомовый территория ...,аварийные деревья,Комитет городского хозяйства города Курска,3,аварийный дерево,Комитет


In [21]:
df_test.head()

,id,Текст Сообщения,Тематика,Ответственное лицо,Тематика Норм,Тип ответственного
0,843,улица мир быть заменить наружное освещение зам...,неработающее наружное освещение,Администрация Курчатовского района,неработающий наружное освещение,Администрация
1,1422,уже второй неделя гореть уличный освещение,неработающее наружное освещение,Комитет жилищно-коммунального хозяйства города...,неработающий наружное освещение,Комитет
2,2782,работать освещение двор дом энергетик световой...,неработающее наружное освещение,Комитет жилищно-коммунального хозяйства города...,неработающий наружное освещение,Комитет
3,2704,покос сорный растительность газон тротуар прое...,неудовлетворительная уборка улиц тротуаров,Администрация Центрального округа города Курска,неудовлетворительный уборка улица тротуар,Администрация
4,1,просить принять мера водитель маршрут пос севе...,неудовлетворительный внешний вид поведение вод...,Администрация города Курска,неудовлетворительный внешний вид поведение вод...,Администрация


## Простейший вариант классификации сравнением Тематики прямым подсчетом совпадающих слов

In [22]:
# Строим словарь соответствий "Множество слов из поля fld_name - Категория" для заданного датасета.

def create_theme_set_cat_map(df: pd.DataFrame, fld_name: str) -> Dict:
    theme_cat = dict()
    for index, row in df.iterrows():
        if row[fld_name] not in theme_cat:
            # theme_cat[frozenset(filter(lambda x: len(x) > 2, row['Тематика'].split()))] = row['Категория']
            theme_cat[frozenset(row[fld_name].split())] = row['Категория']
    return theme_cat

In [23]:
# Подбор категории на основе прямого сравнения множества слов в поле fld_name

def predict_simplest(df_train_data: pd.DataFrame, df_test_data: pd.DataFrame, fld_name: str) -> List[int]:
    theme_set_cat_map = create_theme_set_cat_map(df_train_data, fld_name)
    res_cat = []
    for index, row in df_test_data.iterrows():
        # theme_set = set(filter(lambda x: len(x) > 2, row['Тематика'].split()))
        theme_set = set(row[fld_name].split())
        cur_d = 0
        cur_cat = None
        for key in theme_set_cat_map:
            d = len(key.intersection(theme_set))
            if d > cur_d:
                cur_d = d
                cur_cat = theme_set_cat_map[key]
                # print(key.intersection(theme_set))
        # res_cat.append(cur_cat)
        if cur_d > 0 and ((cur_d <= 3 and len(theme_set) <= 7) or cur_d > 3):
        # if (cur_d == len(theme_set)) or cur_d > 3:
        # if cur_d > 0:
            res_cat.append(cur_cat)
        else:
            res_cat.append(None)
    return res_cat

In [24]:
# Проверка на кроссвалидации варианта с простейшим классификатором. Оценка количества неверно определившихся и не определившихся категорий.

def check_cv_simplest(fld_name: str):
    res_none = []
    res_correct = []
    res_incorrect = []
    for ind in range(50):
        df_train_oper, df_test_oper, train_oper_y, test_oper_y = train_test_split(df_train,
                                                                                  df_train['Категория'], 
                                                                                  test_size=0.3,
                                                                                  random_state=ind * 10)
        predict_y = np.array(predict_simplest(df_train_oper, df_test_oper, fld_name))
        res_none.append(np.sum(predict_y == None))
        res_correct.append(sum((predict_y == test_oper_y)[predict_y != None]))
        res_incorrect.append(sum((predict_y != test_oper_y)[predict_y != None]))
    return dict(res_none=res_none, res_correct=res_correct, res_incorrect=res_incorrect)

def print_stat_check_cv_simplest(r: Dict) -> None:
    print(f"Статистика None. max: {max(r['res_none'])}, min: {min(r['res_none'])}, mean: {np.mean(r['res_none'])}, median: {np.median(r['res_none'])}, sd: {np.std(r['res_none'])}")
    print(f"Статистика неверных ответов. max: {max(r['res_incorrect'])}, min: {min(r['res_incorrect'])}, mean: {np.mean(r['res_incorrect'])}, median: {np.median(r['res_incorrect'])}, sd: {np.std(r['res_incorrect'])}")
    print(f"Статистика верных ответов. max: {max(r['res_correct'])}, min: {min(r['res_correct'])}, mean: {np.mean(r['res_correct'])}, median: {np.median(r['res_correct'])}, sd: {np.std(r['res_correct'])}")


In [25]:
r = check_cv_simplest('Тематика')

In [26]:
print_stat_check_cv_simplest(r)

Статистика None. max: 8, min: 0, mean: 1.42, median: 1.0, sd: 1.5759441614473528
Статистика неверных ответов. max: 6, min: 0, mean: 2.64, median: 3.0, sd: 1.52
Статистика верных ответов. max: 599, min: 586, mean: 595.94, median: 596.0, sd: 2.3444402316971105


In [27]:
r_norm = check_cv_simplest('Тематика Норм')

In [28]:
print_stat_check_cv_simplest(r_norm)

Статистика None. max: 8, min: 0, mean: 1.1, median: 1.0, sd: 1.5394804318340654
Статистика неверных ответов. max: 9, min: 0, mean: 3.08, median: 3.0, sd: 1.9374209661299735
Статистика верных ответов. max: 599, min: 584, mean: 595.82, median: 596.0, sd: 2.76904315603784


Нормализация Тематики почти не повлияла на результат

Не ожидая хороших результатов, применим навскидку этот же подход к полю Текст Сообщения

In [54]:
# Долго работает. Временно комментируем.
# r_txt_msg = check_cv_simplest('Текст Сообщения')

In [30]:
# print_stat_check_cv_simplest(r_txt_msg)

Статистика None. max: 25, min: 9, mean: 17.32, median: 17.0, sd: 3.7333095237336003
Статистика неверных ответов. max: 269, min: 201, mean: 230.38, median: 231.0, sd: 14.673636222831748
Статистика верных ответов. max: 382, min: 309, mean: 352.3, median: 350.5, sd: 14.437797615980077


Как и ожидалось, здесь результаты невысокие.

## Наивный Байес

Используем клкассический вариант алгоритма Наивный Байесовский классификатор. 
Но к традиционным событиям "слово содержится в тексте" добавляем дополнительное событие, позволяющее учесть влияние значения поля Тип ответственного на Категорию.

Под текстом обращения далее понимаем либо текст сообщения, либо текст темы обращения, исходный или преобразованный.

Построим множество слов из текстов всех обращений тренировочного датасета, обозначим его $A$. Пусть $|A| = n$.

Далее выберем и зафиксируем обращение, категорию которого мы хотим определить. Множество слов из текста этого обрашения обозначим $A_0$. Значение поля Тип ответственного этого обращения обозначим $R_0$.

Для произвольного случайно выбранного обращения введем следующие обозначения для событий.

$C_k$ - обращение принадлежит категории $k$.

Для каждого слова $i \in A$ рассмотрим событие $W_i$, которое определяется следующим образом.
Если слово $i \in A_0$, то $W_i$ состоит в том, что слово $i$ содержится в тексте случайно выбранного обращения.
Если слово $i \notin A_0$, то $W_i$ состоит в том, что слово $i$ НЕ содержится в тексте случайно выбранного обращения.

Событие $U$ состоит в том, что значение поля Тип отвественного случайно выбранного события совпадает с $R_0$.

Тогда по теореме Байеса:

$$
P(C_k|W_1\cap W_2\cap ... \cap W_n \cap U) = \frac{P(W_1\cap W_2\cap ... \cap W_n \cap U |C_k)P(C_k)}{P(W_1\cap W_2\cap ... \cap W_n \cap U )}.
$$

Делая наивное предположение о независимости событий, получаем:

$$
P(C_k|W_1\cap W_2\cap ... \cap W_n \cap U) = \frac{P(W_1|C_k)P(W_2|C_k) ... P(W_n|C_k) P(U|C_k)P(C_k)}{P(W_1)P(W_2) ... P(W_n) P(U)}.
$$

Оцениваем шанс попадания зафиксированного события в категорию $k$ как
$$
\frac{P(C_k|W_1\cap W_2\cap ... \cap W_n \cap U)}{\sum_{q = 0}^{16} P(C_q|W_1\cap W_2\cap ... \cap W_n \cap U) }.
$$

И выбираем Категорию, соответствующую максимуму этой величины.

In [31]:
# Универсальный (с точки зрения выбора текстовых полей для классификации) класс для выполнения классификации Наивным Байесом.

class NaiveBayes:
    def __init__(self, train_text_list: List[str], train_resp_type: List[str], train_cat_list: List[int]):
        if not (len(train_text_list) == len(train_cat_list) == len(train_resp_type)):
            raise Exception('Не совпадает длина входных списков')
            
        self.train_cnt = len(train_text_list)
        
        # Подсчитываем в скольких записях встречается каждое слово во всех категориях.
        # Сколько раз встречается каждое слово в каждой категории.
        # Сколько раз встречается каждый Тип ответственного лица в каждой категории
        # А также количество записей с каждой категорией

        self.word_cnt = defaultdict(int)
        self.word_cnt_by_cat = defaultdict(int)
        self.train_cnt_by_cat = defaultdict(int)
        self.resp_type_cnt_by_cat = defaultdict(int)
        for ind in range(len(train_text_list)):
            txt = train_text_list[ind]
            cat = train_cat_list[ind]
            for word in set(txt.split()):
                self.word_cnt[word] += 1
                self.word_cnt_by_cat[(cat, word)] += 1
            self.resp_type_cnt_by_cat[(cat, train_resp_type[ind])] += 1
            self.train_cnt_by_cat[cat] += 1
        
        
    def get_word_cnt_in_cat(self, cat: int, word: str) -> int:
        return self.word_cnt_by_cat.get((cat, word)) or 1

    def get_word_cnt_not_in_cat(self, cat: int, word: str) -> int:
        return (self.train_cnt_by_cat[cat] - self.word_cnt_by_cat.get((cat, word), 0)) or 1
    
    def get_resp_type_cnt_in_cat(self, cat: int, resp_type: str) -> int:
        return self.resp_type_cnt_by_cat.get((cat, resp_type), 0)
        # return self.resp_type_cnt_by_cat.get((cat, resp_type)) or 1
        
    def get_cat_probs(self, txt: str, resp_type: str) -> List[float]:
        chance = [0] * 17
        word_set = set(txt.split())
        for cat in range(17):
            # Если категория встречается слишком редко, то оценки для неё будут содержать большие погрешности
            # Экспериментально подбираем порог, ниже которого метод работать не будет. 
            # В этом случае придется считать, то что такая категория не встречается
            if self.train_cnt_by_cat[cat] < 30:
                chance[cat] = 0
            else:
                chance[cat] = self.train_cnt_by_cat[cat] / self.train_cnt
                for word in self.word_cnt:
                    chance[cat] *= (self.get_word_cnt_in_cat(cat, word) 
                                   if word in word_set else self.get_word_cnt_not_in_cat(cat, word)) / self.train_cnt_by_cat[cat]
                chance[cat] *= self.get_resp_type_cnt_in_cat(cat, resp_type) / self.train_cnt_by_cat[cat]
        sum_chance = sum(chance)
        return [ch / sum_chance for ch in chance], chance

Проверим, что получилось

In [32]:
# Разбиваем тренировочный датасет на две части
df_train_oper, df_test_oper, train_oper_y, test_oper_y = train_test_split(df_train, df_train['Категория'], test_size=0.3, random_state=42)

In [33]:
# Выполняем классификацию, опираясь на данные поля Текст Сообщения

bayes = NaiveBayes(df_train_oper['Текст Сообщения'].values, 
                   df_train_oper['Тип ответственного'].values,
                   df_train_oper['Категория'].values)
b_pred_y = [np.argmax(bayes.get_cat_probs(txt, resp_type)) for txt, resp_type in zip(df_test_oper['Текст Сообщения'].values, 
                                                                          df_test_oper['Тип ответственного'].values)]

In [34]:
sum(test_oper_y != b_pred_y), sum(test_oper_y == b_pred_y)

(350, 250)

In [35]:
# Выполняем классификацию, опираясь на данные поля Тематика

bayes = NaiveBayes(df_train_oper['Тематика'].values, 
                   df_train_oper['Тип ответственного'].values,
                   df_train_oper['Категория'].values)

b_pred_y = [np.argmax(bayes.get_cat_probs(txt, resp_type)) for txt, resp_type in zip(df_test_oper['Тематика'].values, 
                                                                                     df_test_oper['Тип ответственного'].values)]

In [36]:
sum(test_oper_y != b_pred_y), sum(test_oper_y == b_pred_y)

(54, 546)

In [37]:
# Выполняем классификацию, опираясь на данные поля Тематика Норм

bayes = NaiveBayes(df_train_oper['Тематика Норм'].values, 
                   df_train_oper['Тип ответственного'].values,
                   df_train_oper['Категория'].values)

b_pred_y = [np.argmax(bayes.get_cat_probs(txt, resp_type)) for txt, resp_type in zip(df_test_oper['Тематика Норм'].values, 
                                                                                     df_test_oper['Тип ответственного'].values)]

In [38]:
sum(test_oper_y != b_pred_y), sum(test_oper_y == b_pred_y)

(49, 551)

In [39]:
# Проверка на кроссвалидации варианта с наивным Байесом. Оценка количества верно и неверно определившихся.

def check_cv_bayes(fld_name: str) -> Dict:
    res_correct = []
    res_incorrect = []
    for ind in range(50):
        df_train_oper, df_test_oper, train_oper_y, test_oper_y = train_test_split(df_train,
                                                                                  df_train['Категория'], 
                                                                                  test_size=0.3,
                                                                                  random_state=ind * 10)

        bayes = NaiveBayes(df_train_oper[fld_name].values, 
                           df_train_oper['Тип ответственного'].values, 
                           df_train_oper['Категория'].values)
        b_pred_y = [np.argmax(bayes.get_cat_probs(txt, resp_type)) for txt, resp_type in zip(df_test_oper[fld_name],
                                                                                             df_test_oper['Тип ответственного'])]

        res_correct.append(sum((b_pred_y == test_oper_y)))
        res_incorrect.append(sum((b_pred_y != test_oper_y)))
    return dict(res_correct=res_correct, res_incorrect=res_incorrect)

def print_stat_check_cv_bayes(r: Dict) -> None:
    print(f"Статистика неверных ответов. max: {max(r['res_incorrect'])}, min: {min(r['res_incorrect'])}, mean: {np.mean(r['res_incorrect'])}, median: {np.median(r['res_incorrect'])}, sd: {np.std(r['res_incorrect'])}")
    print(f"Статистика верных ответов. max: {max(r['res_correct'])}, min: {min(r['res_correct'])}, mean: {np.mean(r['res_correct'])}, median: {np.median(r['res_correct'])}, sd: {np.std(r['res_correct'])}")

In [40]:
# Опираясь на данные поля Тематика

r_cv_b = check_cv_bayes('Тематика')

In [41]:
print_stat_check_cv_bayes(r_cv_b)

Статистика неверных ответов. max: 82, min: 39, mean: 54.62, median: 54.0, sd: 7.66522015339416
Статистика верных ответов. max: 561, min: 518, mean: 545.38, median: 546.0, sd: 7.665220153394161


In [42]:
# Опираясь на данные поля Тематика Норм

r_cv_b_norm = check_cv_bayes('Тематика Норм')

In [43]:
print_stat_check_cv_bayes(r_cv_b_norm)

Статистика неверных ответов. max: 75, min: 39, mean: 50.5, median: 50.0, sd: 6.9
Статистика верных ответов. max: 561, min: 525, mean: 549.5, median: 550.0, sd: 6.9


In [44]:
# Долго работает. Временно комментируем.
# r_cv_b_msg = check_cv_bayes('Текст Сообщения')

In [45]:
# print_stat_check_cv_bayes(r_cv_b_msg)

Статистика неверных ответов. max: 468, min: 328, mean: 386.6, median: 392.5, sd: 27.13816500797355
Статистика верных ответов. max: 272, min: 132, mean: 213.4, median: 207.5, sd: 27.13816500797355


## Комбинируем оба подхода. Сначала определяем прямым сравнением попадающихся слов в теме. Там где определить не удалось, используем наивный Байес

In [46]:
def predict_comb(df_train_data: pd.DataFrame, df_test_data: pd.DataFrame, print_none_ind=False) -> List[int]:
    predict_y = predict_simplest(df_train_data, df_test_data, 'Тематика')    
    
    bayes = NaiveBayes(df_train_data['Тематика Норм'].values, 
                   df_train_data['Тип ответственного'].values,
                   df_train_data['Категория'].values)
    
    for ind in range(len(predict_y)):
        if predict_y[ind] is None:
            if print_none_ind:
                print(ind)
            predict_y[ind] = np.argmax(bayes.get_cat_probs(df_test_data['Тематика Норм'].values[ind], 
                                                           df_test_data['Тип ответственного'].values[ind]))
    return predict_y

In [47]:
def check_cv_comb() -> Dict:
    res_correct = []
    res_incorrect = []
    for ind in range(50):
        df_train_oper, df_test_oper, train_oper_y, test_oper_y = train_test_split(df_train,
                                                                                  df_train['Категория'], 
                                                                                  test_size=0.3,
                                                                                  random_state=ind * 10)

        b_pred_y = predict_comb(df_train_oper, df_test_oper)
        res_correct.append(sum((b_pred_y == test_oper_y)))
        res_incorrect.append(sum((b_pred_y != test_oper_y)))
    return dict(res_correct=res_correct, res_incorrect=res_incorrect)

def print_stat_check_cv_comb(r: Dict) -> None:
    print(f"Статистика неверных ответов. max: {max(r['res_incorrect'])}, min: {min(r['res_incorrect'])}, mean: {np.mean(r['res_incorrect'])}, median: {np.median(r['res_incorrect'])}, sd: {np.std(r['res_incorrect'])}")
    print(f"Статистика верных ответов. max: {max(r['res_correct'])}, min: {min(r['res_correct'])}, mean: {np.mean(r['res_correct'])}, median: {np.median(r['res_correct'])}, sd: {np.std(r['res_correct'])}")

In [48]:
r_comb = check_cv_comb()

In [49]:
print_stat_check_cv_comb(r_comb)

Статистика неверных ответов. max: 14, min: 1, mean: 3.86, median: 4.0, sd: 2.306599228301267
Статистика верных ответов. max: 599, min: 586, mean: 596.14, median: 596.0, sd: 2.3065992283012666


## Строим прогноз для тестового датасета.

In [50]:
df_test['Категория'] = predict_comb(df_train, df_test, True)

In [51]:
df_final_result = df_test[['id', 'Категория']]

In [52]:
df_final_result.head()

,id,Категория
0,843,3
1,1422,3
2,2782,3
3,2704,3
4,1,8


In [53]:
df_final_result.to_csv('final_result.csv', index = False)